In [27]:
#----- Instalação de bibliotecas necessárias
!pip install unidecode
!pip install rapidfuzz

#----- Importações das bibliotecas
import pandas as pd
import numpy as np
import re
import unidecode
import os
from rapidfuzz import fuzz, process
from IPython.display import display

#----- Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

#----- Caminho base no Google Drive
caminho_base = '/content/drive/My Drive/ProjetoAnalistaDados/'

#----- URLs dos arquivos CSV
urldengue24 = "http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/ae10db21-511e-4295-a027-05af48adf13a/download/dengon-2024.csv"
urlzika24 = "http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/5597d197-b6c8-46af-bad5-473c9f6b3755/download/zika-2024.csv"
urlchikungunya24 = "http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/f6e2e17d-63cd-4998-8bc0-9c19df5b996e/download/chikon-2024.csv"

#----- Caminhos dos arquivos locais no Google Drive
uploaddengue24 = caminho_base + 'Dengue2024_original.csv'
uploadzika24 = caminho_base + 'Zika2024_original.csv'
uploadchikungunya24 = caminho_base + 'Chikungunya2024_original.csv'

#----- Função para carregar CSV com fallback e salvar no Drive
def carregar_csv(url, caminho_local):
    try:
        df = pd.read_csv(url, sep=';', encoding='latin1')
        print(f"✅ Arquivo carregado com sucesso da URL: {url}")
        df.to_csv(caminho_local, sep=';', index=False, encoding='latin1')
        print(f"💾 Cópia salva no Drive em: {caminho_local}")
    except Exception as e:
        print(f"⚠️ Falha ao carregar da URL. Erro: {e}")
        print(f"📂 Tentando carregar do Google Drive: {caminho_local}")
        df = pd.read_csv(caminho_local, sep=';', encoding='latin1')
    return df

#----- Carregando os arquivos com fallback
df_dengue_raw = carregar_csv(urldengue24, uploaddengue24)
df_zika_raw = carregar_csv(urlzika24, uploadzika24)
df_chikungunya_raw = carregar_csv(urlchikungunya24, uploadchikungunya24)



#----- Dicionários e listas
meses_nomes = {1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril', 5: 'Maio', 6: 'Junho',
               7: 'Julho', 8: 'Agosto', 9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'}

dicionario_renomeacao = {
    'nu_notificacao': 'NU_NOTIFIC', 'dt_notificacao': 'DT_NOTIFIC', 'notificacao_ano': 'NU_ANO',
    'ano_notificacao': 'NU_ANO', 'nu_idade': 'NU_IDADE_N', 'tp_sexo': 'CS_SEXO',
    'no_bairro_residencia': 'NM_BAIRRO', 'nome_logradouro_residencia': 'NM_LOGRADO',
    'nm_logradouro_residencia': 'NM_LOGRADO', 'dt_obito': 'DT_OBITO', 'ï»¿nu_notific': 'NU_NOTIFIC'
}

colunas_importantes = ['NU_NOTIFIC', 'DT_NOTIFIC', 'NU_ANO', 'NU_IDADE_N', 'CS_SEXO',
                       'NM_BAIRRO', 'NM_LOGRADO', 'DT_OBITO']

ordem_faixa_etaria = {'Menor de 1 ano': 0, '1 a 9': 1, '10 a 19': 2, '20 a 29': 3, '30 a 39': 4,
                      '40 a 49': 5, '50 a 59': 6, '60 a 69': 7, '70 a 79': 8, '80+': 9}


#Funções de suporte

#----- Função para extrair a idade em anos
def extrair_idade_em_anos(valor):
    try:
        valor_str = str(int(valor)).zfill(4)
        codigo = int(valor_str[0])
        idade = int(valor_str[1:])
        if codigo == 1:
            return round((idade / 24) / 365, 2)
        elif codigo == 2:
            return round(idade / 365, 2)
        elif codigo == 3:
            return round(idade / 12, 2)
        elif codigo == 4:
            return idade
        else:
            return np.nan
    except:
        return np.nan

#----- Função para classificar as idades em faixa etárias
def categorias_faixa_etaria(idade_str):
    try:
        idade = int(idade_str)
    except:
        return 'Sem Informação'
    if idade < 1: return 'Menor de 1 ano'
    elif idade < 10: return '1 a 9'
    elif idade < 20: return '10 a 19'
    elif idade < 30: return '20 a 29'
    elif idade < 40: return '30 a 39'
    elif idade < 50: return '40 a 49'
    elif idade < 60: return '50 a 59'
    elif idade < 70: return '60 a 69'
    elif idade < 80: return '70 a 79'
    else: return '80+'



#----- Função principal que inclui limpeza, normalização e ajustes necessários

def ajustes_df(df, nome):

     # Padroniza os nomes das colunas e remove caracteres estranhos
    df.columns = df.columns.str.upper().str.replace('ï»¿', '')

    # Renomeia colunas conforme dicionário de padronização
    df.rename(columns=dicionario_renomeacao, inplace=True)

    # Filtra apenas colunas relevantes para as análises
    df = df.filter(items=[col for col in colunas_importantes if col in df.columns])

    # Substitui células vazias ou espaços em branco por 'Sem Informação'
    df.replace(r'^\s*$', 'Sem Informação', regex=True, inplace=True)
    df.fillna('Sem Informação', inplace=True)

    # Padroniza a coluna sexo
    if 'CS_SEXO' in df.columns:
        df['CS_SEXO'] = df['CS_SEXO'].map({'F': 'Feminino', 'M': 'Masculino'}).fillna('Indefinido')

    # Substitui idades maiores que 120 por nan, converte idade para número e aplica função para extrair idade em anos


    df['NU_IDADE_N'] = pd.to_numeric(df.get('NU_IDADE_N', pd.Series(dtype='float')), errors='coerce')
    df['IDADE'] = df['NU_IDADE_N'].apply(extrair_idade_em_anos)
    df.loc[df['IDADE'] > 120, 'IDADE'] = np.nan

    # Converte data de notificação para datetime
    df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'], errors='coerce')

    # Extrai mês da notificação e converte para nome do mês
    df['MES_NOTIFIC'] = df['DT_NOTIFIC'].dt.month.map(meses_nomes).fillna("Sem Informação")

    # Cria coluna faixa etária e ordem da faixa
    df['FAIXA_ETARIA'] = df['IDADE'].apply(categorias_faixa_etaria)
    df['ORDEM_FAIXA'] = df['FAIXA_ETARIA'].map(ordem_faixa_etaria)

    # Função para expandir abreviações mais comuns
    def expandir_abreviacoes(logradouro):
        substituicoes = {
            r'\bAV\b\.?': 'AVENIDA', r'\bR\b\.?': 'RUA', r'\bDR\b\.?': 'DOUTOR',
            r'\bPROF\b\.?': 'PROFESSOR', r'\bVER\b\.?': 'VEREADOR',
            r'\bTRAV\b\.?': 'TRAVESSA', r'\bTV\b\.?': 'TRAVESSA',
            r'\bPRACA\b\.?': 'PRACA'
        }
        for abrev, completo in substituicoes.items():
            logradouro = re.sub(abrev, completo, logradouro)
        return logradouro

    # Função para remover preposições desnecessárias
    def remover_preposicoes(texto):
        preposicoes = [' DE ', ' DA ', ' DO ', ' DAS ', ' DOS ']
        for prep in preposicoes:
            texto = texto.replace(prep, ' ')
        return re.sub(r'\s+', ' ', texto).strip()

    # Função geral de padronização que aplica as anteriores
    def padronizar_logradouro(logradouro):
        if pd.isna(logradouro): return ""
        logradouro = unidecode.unidecode(logradouro.upper())
        logradouro = expandir_abreviacoes(logradouro)
        logradouro = remover_preposicoes(logradouro)
        return logradouro.strip()

    # Aplica a padronização de logradouros
    df['NM_LOGRADO'] = df['NM_LOGRADO'].astype(str).apply(padronizar_logradouro)
     # Correção manual de um caso específico
    df['NM_LOGRADO'] = df['NM_LOGRADO'].replace({
        'AVENIDA VEREADOR OTACILIO AZEVEDO': 'AVENIDA VEREADOR OTACILIO'
    })

    # Padroniza logradouros semelhantes com fuzzy matching
    def padronizar_logradouros_fuzzy(df, coluna='NM_LOGRADO', limite_score=85):
        logradouro_mapeado, logradouros_padronizados = {}, []
        for log in df[coluna].dropna().unique():
            if log in logradouro_mapeado:
                continue
            resultado = process.extractOne(log, logradouros_padronizados, scorer=fuzz.token_sort_ratio)
            if resultado:
                match, score, _ = resultado
                logradouro_mapeado[log] = match if score >= limite_score else log
            else:
                logradouro_mapeado[log] = log
            logradouros_padronizados.append(logradouro_mapeado[log])
        df[coluna] = df[coluna].map(logradouro_mapeado)
        return df

    df = padronizar_logradouros_fuzzy(df)
    return df

#----- Processamento dos dados
dfdengue24 = ajustes_df(df_dengue_raw, 'dfdengue24')
dfzika24 = ajustes_df(df_zika_raw, 'dfzika24')
dfchikungunya24 = ajustes_df(df_chikungunya_raw, 'dfchikungunya24')

#----- Adiciona tipo de doença
dfdengue24['TIPO_DOENCA'] = 'Dengue'
dfzika24['TIPO_DOENCA'] = 'Zika'
dfchikungunya24['TIPO_DOENCA'] = 'Chikungunya'

#----- Junta os dados
df_unidos = pd.concat([dfdengue24, dfzika24, dfchikungunya24])

#----- Cria coluna de óbito
df_unidos['ÓBITO'] = df_unidos['DT_OBITO'].apply(lambda x: 0 if x == 'Sem Informação' or pd.isna(x) else 1)

#-----Análises Exploratórias

# Quantidade total de registros e colunas
print(f"Total de registros: {df_unidos.shape[0]}")
print(f"Total de colunas: {df_unidos.shape[1]}")

#-----#Estatísticas de Outliers

# Cálculo do IQR
Q1 = df_unidos['IDADE'].quantile(0.25)
Q3 = df_unidos['IDADE'].quantile(0.75)
IQR = Q3 - Q1

# Limites inferior e superior
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar outliers
outliers = df_unidos[(df_unidos['IDADE'] < limite_inferior) | (df_unidos['IDADE'] > limite_superior)]

print(f"Quantidade de outliers na idade: {len(outliers)}")
print(outliers[['IDADE']].describe())


#Estatísticas Descritivas Categóricas

display(df_unidos[['TIPO_DOENCA', 'CS_SEXO', 'FAIXA_ETARIA', 'NM_BAIRRO']].describe(include='object'))


#Casos por mês

display(df_unidos['MES_NOTIFIC'].value_counts().reindex(list(meses_nomes.values())))

# Proporção de óbitos
proporcao_obitos = df_unidos['ÓBITO'].mean()
print(f"Proporção de óbitos: {proporcao_obitos:.2%}")

# Por tipo de doença
df_unidos.groupby('TIPO_DOENCA')['ÓBITO'].mean().apply(lambda x: f"{x:.2%}")

#Idade Média e Mediana por Doença
df_unidos.groupby('TIPO_DOENCA')['IDADE'].agg(['mean', 'median']).round(1)


#Top 10 bairros e logradouros com mais casos

top_bairros = df_unidos['NM_BAIRRO'].value_counts().head(10)
top_logradouros = df_unidos['NM_LOGRADO'].value_counts().head(10)
print("Top 10 bairros:\n", top_bairros)
print("\nTop 10 logradouros:\n", top_logradouros)

# Análise de correlações

#Gênero x tipo de doença

genero_doenca = pd.crosstab(df_unidos['CS_SEXO'], df_unidos['TIPO_DOENCA'], normalize='index') * 100
print("Gênero vs Tipo de Doença (% por linha):")
print(genero_doenca.round(2))


#Faixa etária x tipo de doença

faixa_doenca = pd.crosstab(df_unidos['FAIXA_ETARIA'], df_unidos['TIPO_DOENCA'], normalize='index') * 100
faixa_doenca = faixa_doenca.reindex(sorted(faixa_doenca.index, key=lambda x: ordem_faixa_etaria.get(x, 999)))

print("Faixa Etária vs Tipo de Doença (% por linha):")
print(faixa_doenca.round(2))

#----- Exporta os dados para o Google Drive
os.makedirs(caminho_base, exist_ok=True)
dfdengue24.to_csv(f"{caminho_base}dfdengue24_padronizado3.csv", index=False, encoding='utf-8-sig', sep=';')
dfzika24.to_csv(f"{caminho_base}dfzika24_padronizado3.csv", index=False, encoding='utf-8-sig', sep=';')
dfchikungunya24.to_csv(f"{caminho_base}dfchikungunya24_padronizado3.csv", index=False, encoding='utf-8-sig', sep=';')
df_unidos.to_csv(f"{caminho_base}dfunidos4.csv", index=False, encoding='utf-8-sig', sep=';')
print("✅ Arquivos salvos com sucesso no Google Drive!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-27-4754eca8f3de>:34: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url, sep=';', encoding='latin1')


✅ Arquivo carregado com sucesso da URL: http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/ae10db21-511e-4295-a027-05af48adf13a/download/dengon-2024.csv
💾 Cópia salva no Drive em: /content/drive/My Drive/ProjetoAnalistaDados/Dengue2024_original.csv
✅ Arquivo carregado com sucesso da URL: http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/5597d197-b6c8-46af-bad5-473c9f6b3755/download/zika-2024.csv
💾 Cópia salva no Drive em: /content/drive/My Drive/ProjetoAnalistaDados/Zika2024_original.csv
✅ Arquivo carregado com sucesso da URL: http://dados.recife.pe.gov.br/dataset/2a9b1c39-0700-4ddf-9a10-b3c8d5d9396c/resource/f6e2e17d-63cd-4998-8bc0-9c19df5b996e/download/chikon-2024.csv
💾 Cópia salva no Drive em: /content/drive/My Drive/ProjetoAnalistaDados/Chikungunya2024_original.csv
Total de registros: 13414
Total de colunas: 14
Quantidade de outliers na idade: 94
            IDADE
count   94.000000
mean    83.361702
std      4.86340

,TIPO_DOENCA,CS_SEXO,FAIXA_ETARIA,NM_BAIRRO
count,13414,13414,13414,13414
unique,3,3,11,94
top,Dengue,Feminino,20 a 29,IBURA
freq,10458,7545,3155,947


,count
MES_NOTIFIC,
Janeiro,368
Fevereiro,1581
Março,3026
Abril,2596
Maio,1146
Junho,1126
Julho,920
Agosto,738
Setembro,441


Proporção de óbitos: 0.29%
Top 10 bairros:
 NM_BAIRRO
IBURA               947
COHAB               730
VARZEA              677
NOVA DESCOBERTA     584
BOA VIAGEM          564
IPUTINGA            560
IMBIRIBEIRA         444
JARDIM SAO PAULO    383
SANTO AMARO         323
VASCO DA GAMA       323
Name: count, dtype: int64

Top 10 logradouros:
 NM_LOGRADO
AVENIDA VEREADOR OTACILIO       127
AVENIDA PROFESSOR JOSE ANJOS     32
RUA SAO MATEUS                   31
AVENIDA BOA VIAGEM               27
RUA NOSSA SENHORA CARMO          26
AVENIDA BEBERIBE                 25
RUA RIO                          24
AVENIDA MARECHAL MASC MORAIS     24
RUA SAO SEBASTIAO                23
RUA CALCADAS                     23
Name: count, dtype: int64
Gênero vs Tipo de Doença (% por linha):
TIPO_DOENCA  Chikungunya  Dengue  Zika
CS_SEXO                               
Feminino           19.73   76.24  4.03
Indefinido         12.00   88.00  0.00
Masculino          16.94   80.12  2.94
Faixa Etária vs Tipo de Do